# Imports

In [5]:
from datetime import datetime,timedelta
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from dateutil import parser
import os
import config
from jugaad_trader import Zerodha
import pyotp
from pymongo import MongoClient

# Login to Kite

In [2]:
username = config.username
password = config.password
totp = pyotp.TOTP(config.totp_code)
kite = Zerodha(user_id=username, password=password, twofa=totp.now())
kite.login()

In [ ]:
# df_instruments=pd.DataFrame(kite.instruments())
# df_instruments.to_csv("instruments_all.csv", index=False)

# Utility Functions

In [ ]:
def get_candle_fig(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        fig = go.Figure(data=[go.Candlestick(x=df_historical['date'],
                        open=df_historical['open'],
                        high=df_historical['high'],
                        low=df_historical['low'],
                        close=df_historical['close'])])
        return (fig)
    else:
        return (None)
    
def get_candle(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        return (df_historical)
    else:
        return (pd.DataFrame())
    
def get_candle_signal(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return ((sum(df_summary['candle_ratio'])*2)/sample_size)
    except:
        return 0
    
def get_candle_signal_2(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return (len(df_summary[df_summary['candle_ratio']>0.5])/len(df_summary))
    except:
        return 0
    
def get_candle_signal_3(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:1 if x>0 else -1)
                df_historical['candle_weight']=df_historical['change'].abs()/df_historical['change'].abs().max()
                df_historical['candle_strength']=df_historical['candle_type']*df_historical['candle_weight']
                try:
                    candle_ratio= sum(df_historical['candle_strength'])
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return ((sum(df_summary['candle_ratio']))/sample_size)
    except:
        return 0

def shortlist_scrips():
    df_instruments1=pd.read_excel('final_instruments.xlsx')
    df_instruments1['signal_3_1d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,1))
    df_instruments1['signal_3_7d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,7))
    df_instruments1['signal_3_31d']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_3(x,31))
    df_instruments1=df_instruments1.sort_values(by="signal_3_31d",ascending=False).reset_index(drop=True)
    date=datetime.now().strftime("%d_%m_%y")
    filename=f'weighted_final_shortlist_{date}.csv'
    if filename not in os.listdir():
        df_instruments1.to_csv(filename,index=False)
    df_instruments1=pd.read_csv(filename)
    buy_shortlist_df=df_instruments1[(df_instruments1['signal_3_1d']>0)&(df_instruments1['signal_3_7d']>0)&(df_instruments1['signal_3_31d']>0)]
    sell_shortlist_df=df_instruments1[(df_instruments1['signal_3_1d']<0)&(df_instruments1['signal_3_7d']<0)&(df_instruments1['signal_3_31d']<0)]
    return (buy_shortlist_df, sell_shortlist_df)

In [ ]:
buy_shortlist_df, sell_shortlist_df=shortlist_scrips()

# Stop Loss Multiplier Strategy

In [ ]:
def sl_multiplier_strategy(instrument_token,multiplier):
    '''
    The strategy is to read the latest available candle(reference candle/RC) 
    (Probably 9:15-9:20 AM or 9:15-9:30 AM) and place the order in the second candle based on following logics:
    
    1. Order Price: Price at which we enter the market with buy or sell order. 
    The order price is equal to the closing price of RC. 

    2. Order Type: It is either buy/sell. If the RC is green, we create a buy order or if it's red we create a sell order.

    3. Stop Loss: It is equal to the low of the RC.

    4. Target Price: Exit point of trade. It is [('multiplier' x (order price - stoploss) in case of buy order)/('multiplier' x (stoploss - order price ) in case of sell order)]
    
    '''
    start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:15:00"
    end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 15:30:00"
    interval="5minute"
    df_candle=get_candle(instrument_token,start,end,interval)
    df_candle['date']=df_candle['date'].apply(lambda x: parser.parse(str(x).replace('+05:30','')))
    df_candle['day']=df_candle['date'].apply(lambda x: x.date())
    df_candle['change']=df_candle['close']-df_candle['open']
    order_price=df_candle.loc[len(df_candle)-1,'close']
    order_type="buy"
    stop_loss=df_candle.loc[len(df_candle)-1,'low']
    target_price=order_price+((order_price-stop_loss)*multiplier)
    order = {'instrument_code':instrument_token,
            'order_price':order_price,
            'order_type':order_type,
            'stop_loss':stop_loss,
            'target_price':target_price}
    return order


In [ ]:
## Backtesting the SL Multiplier Strategy

sample_size=1
orders=[]
for i2,row2 in buy_shortlist_df.iterrows():
    instrument_code=row2['instrument_token']
    df_candle=pd.DataFrame()
    for i in range(sample_size):
        start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:15:00"
        end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 15:30:00"
        interval="minute"
        df_historical=get_candle(instrument_code,start,end,interval)
        df_candle=pd.concat([df_historical,df_candle])
    df_candle['date']=df_candle['date'].apply(lambda x: parser.parse(str(x).replace('+05:30','')))
    df_candle['day']=df_candle['date'].apply(lambda x: x.date())
    df_candle['change']=df_candle['close']-df_candle['open']
    for day in df_candle['day'].unique():
        filter_df=df_candle[df_candle['day']==day].reset_index(drop=True)
        book_price=None
        for (i,row) in filter_df.iterrows():
            if ((i==0)&(row['change']>0)):
                order_price=row['close']
                order_type="buy"
                stop_loss=row['low']
                target_price=order_price+((order_price-stop_loss)*3)
            elif ((i==0)&(row['change']<0)):
                order_price=row['close']
                order_type="sell"
                stop_loss=row['high']
                target_price=order_price-((stop_loss-order_price)*3)
            elif ((i==0)&(row['change']==0)):
                pass
            else:
                if (((order_type=="sell")&(row['open']<=target_price))|((order_type=="sell")&(row['open']>=stop_loss))):
                    book_price=row['open']
                    book_time=row['date']
                    orders.append({'instrument_code':instrument_code,
                                    'order_price':order_price,
                                    'order_type':order_type,
                                    'stop_loss':stop_loss,
                                    'target_price':target_price,
                                    'book_price':book_price,
                                    'book_time':book_time})
                    break
                elif (((order_type=="buy")&(row['open']>=target_price))|((order_type=="buy")&(row['open']<=stop_loss))):
                    book_price=row['open']
                    book_time=row['date']
                    orders.append({'instrument_code':instrument_code,
                                    'order_price':order_price,
                                    'order_type':order_type,
                                    'stop_loss':stop_loss,
                                    'target_price':target_price,
                                    'book_price':book_price,
                                    'book_time':book_time})
                    break
                else:
                    pass
        if book_price is None:
            book_time=filter_df.loc[len(filter_df)-10,'date']
            book_price=filter_df.loc[len(filter_df)-10,'open']
            orders.append({'instrument_code':instrument_code,
                                    'order_price':order_price,
                                    'order_type':order_type,
                                    'stop_loss':stop_loss,
                                    'target_price':target_price,
                                    'book_price':book_price,
                                    'book_time':book_time})
            
orders_df=pd.DataFrame(orders)
orders_df.to_clipboard()



In [ ]:
#Unit Testing

sample_size=1
instrument_code=2977281
df_candle=pd.DataFrame()
orders=[]
for i in range(sample_size):
    start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:15:00"
    end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 15:30:00"
    interval="5minute"
    df_historical=get_candle(instrument_code,start,end,interval)
    df_candle=pd.concat([df_historical,df_candle])
df_candle['date']=df_candle['date'].apply(lambda x: parser.parse(str(x).replace('+05:30','')))
df_candle['day']=df_candle['date'].apply(lambda x: x.date())
df_candle['change']=df_candle['close']-df_candle['open']
for day in df_candle['day'].unique():
    filter_df=df_candle[df_candle['day']==day].reset_index(drop=True)
    book_price=None
    for (i,row) in filter_df.iterrows():
        if ((i==0)&(row['change']>0)):
            order_price=row['close']
            order_type="buy"
            stop_loss=row['low']
            target_price=order_price+((order_price-stop_loss)*3)
        elif ((i==0)&(row['change']<0)):
            order_price=row['close']
            order_type="sell"
            stop_loss=row['high']
            target_price=order_price-((stop_loss-order_price)*3)
        elif ((i==0)&(row['change']==0)):
            pass
        else:
            if (((order_type=="sell")&(row['open']<=target_price))|((order_type=="sell")&(row['open']>=stop_loss))):
                book_price=row['open']
                book_time=row['date']
                orders.append({'instrument_code':instrument_code,
                                'order_price':order_price,
                                'order_type':order_type,
                                'stop_loss':stop_loss,
                                'target_price':target_price,
                                'book_price':book_price,
                                'book_time':book_time})
                break
            elif (((order_type=="buy")&(row['open']>=target_price))|((order_type=="buy")&(row['open']<=stop_loss))):
                book_price=row['open']
                book_time=row['date']
                orders.append({'instrument_code':instrument_code,
                                'order_price':order_price,
                                'order_type':order_type,
                                'stop_loss':stop_loss,
                                'target_price':target_price,
                                'book_price':book_price,
                                'book_time':book_time})
                break
            else:
                pass
    if book_price is None:
        book_time=filter_df.loc[len(filter_df)-10,'date']
        book_price=filter_df.loc[len(filter_df)-10,'open']
        orders.append({'instrument_code':instrument_code,
                                'order_price':order_price,
                                'order_type':order_type,
                                'stop_loss':stop_loss,
                                'target_price':target_price,
                                'book_price':book_price,
                                'book_time':book_time})

In [10]:
client=MongoClient()
db=client['kite']
collection=db['tick_data']
record=collection.find({'instrument_token': 738561 }).sort({'exchange_timestamp': -1}).limit(1)

In [12]:
for price_data in record:
    print("Latest price for AAPL:", price_data["instrument_token"])

Latest price for AAPL: 738561


In [11]:
record

In [ ]:
# Place an order
order_id = kite.place_order(tradingsymbol="INFY",
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=1,
                            variety=kite.VARIETY_AMO,
                            order_type=kite.ORDER_TYPE_MARKET,
                            product=kite.PRODUCT_CNC,
                            validity=kite.VALIDITY_DAY)



# Fetch all orders
kite.orders()

# Get instruments
kite.instruments()

# Place an mutual fund order
kite.place_mf_order(
    tradingsymbol="INF090I01239",
    transaction_type=kite.TRANSACTION_TYPE_BUY,
    amount=5000,
    tag="mytag"
)

# Cancel a mutual fund order
kite.cancel_mf_order(order_id="order_id")

# Get mutual fund instruments
kite.mf_instruments()

In [ ]:
###############################################################################
#
# The MIT License (MIT)
#
# Copyright (c) Zerodha Technology Pvt. Ltd.
#
# This example shows how to run KiteTicker in threaded mode.
# KiteTicker runs in seprate thread and main thread is blocked to juggle between
# different modes for current subscribed tokens. In real world web apps
# the main thread will be your web server and you can access WebSocket object
# in your main thread while running KiteTicker in separate thread.
###############################################################################

import time
import logging
from kiteconnect import KiteTicker

logging.basicConfig(level=logging.DEBUG)

# Initialise.
kws = KiteTicker(key, token)

# RELIANCE BSE
tokens = ce_shortlist_df['instrument_token'].tolist()


# Callback for tick reception.
def on_ticks(ws, ticks):
    if len(ticks) > 0:
        logging.info("Current mode: {}".format(ticks[0]["mode"]))
        for tick in ticks:
            insert_tick(tick)


# Callback for successful connection.
def on_connect(ws, response):
    logging.info("Successfully connected. Response: {}".format(response))
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, tokens)
    logging.info("Subscribe to tokens in Full mode: {}".format(tokens))


# Callback when current connection is closed.
def on_close(ws, code, reason):
    logging.info("Connection closed: {code} - {reason}".format(code=code, reason=reason))


# Callback when connection closed with error.
def on_error(ws, code, reason):
    logging.info("Connection error: {code} - {reason}".format(code=code, reason=reason))


# Callback when reconnect is on progress
def on_reconnect(ws, attempts_count):
    logging.info("Reconnecting: {}".format(attempts_count))


# Callback when all reconnect failed (exhausted max retries)
def on_noreconnect(ws):
    logging.info("Reconnect failed.")


# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_close = on_close
kws.on_error = on_error
kws.on_connect = on_connect
kws.on_reconnect = on_reconnect
kws.on_noreconnect = on_noreconnect

# Infinite loop on the main thread.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect(threaded=True)

# Block main thread
logging.info("This is main thread. Will change webosocket mode every 5 seconds.")

# count = 0
# while True:
#     count += 1
#     if count % 2 == 0:
#         if kws.is_connected():
#             logging.info("### Set mode to LTP for all tokens")
#             kws.set_mode(kws.MODE_LTP, tokens)
#     else:
#         if kws.is_connected():
#             logging.info("### Set mode to quote for all tokens")
#             kws.set_mode(kws.MODE_QUOTE, tokens)

#     time.sleep(5)


In [ ]:
a=5

In [ ]:
aa=33

In [ ]:
import pyotp
import time
import config

totp = pyotp.TOTP(config.totp_code)
totp.now() # => '492039'

In [ ]:
from kiteconnect import KiteConnect
import config
import pyotp
api_key = config.api_key
api_secret = config.api_secret
username = config.username
password = config.password
totp = pyotp.TOTP(config.totp_code)

# Initialize Kite Connect object
kite = KiteConnect(api_key=api_key)

# Get the login URL
login_url = kite.login_url()

# Generate the session access token
session_data = kite.generate_session(username, password, api_secret, totp)
access_token = session_data['access_token']

# Set the access token
kite.set_access_token(access_token)